In [1]:
import netCDF4 as nc4
import xarray as xr
from icecream import ic
import subprocess
ic.configureOutput(prefix="", outputFunction=print)

In [2]:
ic(nc4.__version__)
ic(xr.__version__)
print("")

nc4.__version__: '1.5.3'
xr.__version__: '0.20.2'



In [3]:
dummy_base = list(range(5))
dummy_base[2] = -99.
dummy_data = [1.0, 2.0, -99., 4.0, -99.]

In [4]:
path_initial_nc = "initial_nc.nc"

with nc4.Dataset(path_initial_nc, "w", format="NETCDF4") as nc4_out:
    dummy_base_dim = nc4_out.createDimension("dummy_base", len(dummy_base))
    dummy_base_var = nc4_out.createVariable("dummy_base", "f4", ("dummy_base"), fill_value=-99.)
    dummy_var = nc4_out.createVariable("dummy_variable", "f4", ("dummy_base"), fill_value=-99.)

    dummy_base_var[:] = dummy_base
    dummy_var[:] = dummy_data

```
> ncdump -h initial_nc.nc 
netcdf initial_nc {
dimensions:
	dummy_base = 5 ;
variables:
	float dummy_base(dummy_base) ;
		dummy_base:_FillValue = -99.f ;
	float dummy_variable(dummy_base) ;
		dummy_variable:_FillValue = -99.f ;
}
```

In [5]:
xr_initial = xr.load_dataset(path_initial_nc)

In [6]:
xr_initial

<xarray.Dataset>
Dimensions:         (dummy_base: 5)
Coordinates:
  * dummy_base      (dummy_base) float32 0.0 1.0 nan 3.0 4.0
Data variables:
    dummy_variable  (dummy_base) float32 1.0 2.0 nan 4.0 nan

In [7]:
ic(xr_initial["dummy_base"].data)
ic(xr_initial["dummy_variable"].data)
print("")

xr_initial["dummy_base"].data: array([ 0.,  1., nan,  3.,  4.], dtype=float32)
xr_initial["dummy_variable"].data: array([ 1.,  2., nan,  4., nan], dtype=float32)



In [8]:
path_dumped_xr = "dumped_nc_through_xr.nc"

xr_initial.to_netcdf(path_dumped_xr)

In [9]:
with nc4.Dataset(path_dumped_xr, "r", format="NETCDF4") as nc4_in:
    loaded_base = nc4_in["dummy_base"][:]
    loaded_variable = nc4_in["dummy_variable"][:]

ic(loaded_base)
ic(loaded_variable)
print("")

loaded_base: masked_array(data=[0.0, 1.0, --, 3.0, 4.0],
                          mask=[False, False,  True, False, False],
                    fill_value=-99.0,
                         dtype=float32)
loaded_variable: masked_array(data=[1.0, 2.0, --, 4.0, --],
                              mask=[False, False,  True, False,  True],
                        fill_value=-99.0,
                             dtype=float32)



<ipython-input-9-b9bffaf629e2>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  loaded_base = nc4_in["dummy_base"][:]
<ipython-input-9-b9bffaf629e2>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  loaded_variable = nc4_in["dummy_variable"][:]


```
> ncdump -h dumped_nc_through_xr.nc 
netcdf dumped_nc_through_xr {
dimensions:
	dummy_base = 5 ;
variables:
	float dummy_base(dummy_base) ;
		dummy_base:_FillValue = -99.f ;
	float dummy_variable(dummy_base) ;
		dummy_variable:_FillValue = -99.f ;
}
```